In [6]:
from collections import defaultdict

class State:
    def __init__(self):
        self.register = defaultdict(lambda: 0)
        self.frequency = 0
        self.ip = 0

def snd_op(operand):
    try:
        op = int(operand)
        def snd(state):
            state.frequency = op
    except ValueError:
        def snd(state):
            state.frequency = state.register[operand]
    return snd


def set_op(dest, operand):
    try:
        op = int(operand)
        def set(state):
            state.register[dest] = op
    except ValueError:
        def set(state):
            state.register[dest] = state.register[operand]
    return set

        
def add_op(dest, operand):
    try:
        op = int(operand)
        def add(state):
            state.register[dest] += op
    except ValueError:
        def add(state):
            state.register[dest] += state.register[operand]
    return add

    
def mul_op(dest, operand):
    try:
        op = int(operand)
        def mul(state):
            state.register[dest] *= op
    except ValueError:
        def mul(state):
            state.register[dest] *= state.register[operand]
    return mul


def mod_op(dest, operand):
    try:
        op = int(operand)
        def mod(state):
            state.register[dest] %= op
    except ValueError:
        def mod(state):
            state.register[dest] %= state.register[operand]
    return mod


def rcv_op(operand):
    return lambda state: state.frequency if state.register[operand] else None
    

def jgz_op(condition, operand):
    try:
        cond = int(condition)
        cond = lambda state: cond
    except ValueError:
        cond = lambda state: state.register[condition]
    try:
        operand = int(operand)
        op = lambda state: operand
    except ValueError:
        op = lambda state: state.register[operand]
    def jgz(state):
        if cond(state):
            state.ip += op(state) - 1
    return jgz

    
def parse(lines):
    operations = {
        'snd': snd_op,
        'set': set_op,
        'add': add_op,
        'mul': mul_op,
        'mod': mod_op,
        'rcv': rcv_op,
        'jgz': jgz_op,
    }
    program = []
    for line in lines:
        tokens = line.split()
        program.append(operations[tokens[0]](*tokens[1:]))
    return program


def execute(program):
    state = State()
    while True:
        #print(state.ip + 1)
        #print(program[state.ip])
        #print(state.register)
        result = program[state.ip](state)
        if result:
            return result
        state.ip += 1

        
test_input = """set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
"""
p1 = parse(test_input.splitlines())
#execute(p1)

with open('input', 'r') as f:
    p2 = parse(f)
    print(execute(p2))

1187


In [1]:
from collections import defaultdict
from queue import Queue
from threading import Thread

class State:
    def __init__(self):
        self.register = defaultdict(lambda: 0)
        self.ip = 0
        self.inbox = Queue()
        self.outbox = None
        self.counter = 0


def snd_op(operand):
    try:
        op = int(operand)
        def snd(state):
            state.outbox.put(op)
            state.counter += 1
    except ValueError:
        def snd(state):
            state.outbox.put(state.register[operand])
            state.counter += 1
    return snd


def set_op(dest, operand):
    try:
        op = int(operand)
        def set(state):
            state.register[dest] = op
    except ValueError:
        def set(state):
            state.register[dest] = state.register[operand]
    return set

        
def add_op(dest, operand):
    try:
        op = int(operand)
        def add(state):
            state.register[dest] += op
    except ValueError:
        def add(state):
            state.register[dest] += state.register[operand]
    return add

    
def mul_op(dest, operand):
    try:
        op = int(operand)
        def mul(state):
            state.register[dest] *= op
    except ValueError:
        def mul(state):
            state.register[dest] *= state.register[operand]
    return mul


def mod_op(dest, operand):
    try:
        op = int(operand)
        def mod(state):
            state.register[dest] %= op
    except ValueError:
        def mod(state):
            state.register[dest] %= state.register[operand]
    return mod


def rcv_op(operand):
    def rcv(state):
        state.register[operand] = state.inbox.get()
    return rcv
    

def jgz_op(condition, operand):
    try:
        cond = int(condition)
        cond = lambda state: cond
    except ValueError:
        cond = lambda state: state.register[condition]
    try:
        operand = int(operand)
        op = lambda state: operand
    except ValueError:
        op = lambda state: state.register[operand]
    def jgz(state):
        if cond(state):
            state.ip += op(state) - 1
    return jgz

    
def parse(lines):
    operations = {
        'snd': snd_op,
        'set': set_op,
        'add': add_op,
        'mul': mul_op,
        'mod': mod_op,
        'rcv': rcv_op,
        'jgz': jgz_op,
    }
    program = []
    for line in lines:
        tokens = line.split()
        program.append(operations[tokens[0]](*tokens[1:]))
    return program


class Program(Thread):
    def __init__(self, program, state):
        self._program = program
        self._state = state
        super().__init__()
    
    def run(self):
        while True:
            #print(self._program[self._state.ip])
            #print("({})".format(self._state.ip))
            self._program[self._state.ip](self._state)
            self._state.ip += 1
            assert(self._state.ip >= 0)

        
def run_both(program):
    state0 = State()
    state1 = State()
    state0.outbox = state1.inbox
    state1.outbox = state0.inbox
    state0.register['p'] = 0
    state1.register['p'] = 1
    t0 = Program(program, state0)
    t1 = Program(program, state1)
    t0.start()
    t1.start()
    t0.join()
    t1.join()
    return state1.count
    
    
test_input = """snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
"""
#p1 = parse(test_input.splitlines())
#print(p1)
#run_both(p1)

with open('input', 'r') as f:
    p2 = parse(f)
    print(run_both(p2))

KeyboardInterrupt: 